In [3]:
import shutil
import json
from PIL import Image
import os
import shutil
import random
from collections import Counter
from collections import defaultdict
random.seed(418)
pwd = os.getcwd().replace("\\","/")
print(pwd)

"""
Synthetic Mathematical Dataset
Real-World Dataset
"""
r = 0
index = 1
file_names = ["Synthetic Mathematical Dataset", "Real-World Dataset"]
json_names = ["/Synthetic.json", "/Real.json"]
file_name = file_names[index]
json_name= json_names[index]

sou = "/".join(pwd.split("/")[:5]) + "/"
im = "/".join(pwd.split("/")[:4]) + "/"
print(im)

# 定义源目录和目标目录
source_dir = sou + file_name
# target_dir = pwd + "/" + file_name + "/images"
target_dir = pwd + "/" + file_name

output_path = pwd + "/" + file_name + json_name

# 检查目标目录是否存在
if os.path.exists(target_dir):
    # 删除目标目录及其所有内容
    shutil.rmtree(target_dir)
    print(f"Directory '{target_dir}' exists. Deleted all contents.")
# 创建一个新的空目录
os.makedirs(target_dir)
print(f"Directory '{target_dir}' has been created.")

def get_number_dict():
    number_dict = {"Length": 0,
            "Quantity": 0,
            "Angle": 0,
            "Volume": 0,
            "Scale": 0,
            "Depth": 0,
            "Area": 0}
    # 遍历源目录及子目录
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if file.endswith(('.json')):
                # 获取源文件路径
                source_file = os.path.join(root, file)
                if "init_image" in source_file:
                    continue
                with open(source_file, 'r', encoding='utf-8') as file:
                    data = json.load(file)
                for an in data:
                    cls = an["image_path"].split("/")[2].split("_")[0]
                    source_image = os.path.join(im, an["image_path"])
                    if not os.path.exists(source_image):
                        continue
                    number_dict[cls] += 1

    # 将字典的值转换为随机顺序的数组
    for key in number_dict:
        # 创建一个从 0 到 number_dict[key] 的列表
        array = list(range(number_dict[key]))  # 注意这里要加1，因为 range 不包含结束值
        # 打乱列表
        random.shuffle(array)
        # 将打乱后的列表赋值给字典的值
        number_dict[key] = array
    return number_dict
def resize_image(source_file, target_file):
    # 打开图片
    with Image.open(source_file) as img:
                # Convert the image to RGB mode
        img = img.convert("RGB")
        # 获取图片的尺寸 (宽度, 高度)
        w, h = img.size
        
        # 如果图片的宽度或高度大于500，进行缩放
        if w > 500 or h > 500:
            # 计算缩放比例
            if w > h:
                new_w = 500
                new_h = int((500 / w) * h)
            else:
                new_h = 500
                new_w = int((500 / h) * w)
            
            # 缩放图片
            img = img.resize((new_w, new_h), Image.Resampling.LANCZOS)
        
        # 保存到新路径
        img.save(target_file, format='PNG')
        
all_json_list = []
number = 0
all_data = []
number_dict = {"Length": 0,
        "Quantity": 0,
        "Angle": 0,
        "Volume": 0,
        "Scale": 0,
        "Depth": 0,
        "Area": 0}
number_key = get_number_dict()
print(source_dir)
# 遍历源目录及子目录
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if file.endswith(('.json')):
            # 获取源文件路径
            source_file = os.path.join(root, file)
            if "init_image" in source_file:
                continue
            with open(source_file, 'r', encoding='utf-8') as file:
                data = json.load(file)
            for an in data:
                source_image = os.path.join(im, an["image_path"])
                # print(source_image)
                if not os.path.exists(source_image):
                    continue
                cls = an["image_path"].split("/")[2].split("_")[0]
                # if cls != "Angle":
                #     continue
                
                # if cls != "Depth":
                #     continue
                an["Major_categories"] = cls
                an["Minor_category"] = an["image_path"].split("/")[3]
                #hours
                option_text = an["option"].replace("10 hours", "wengtengjin").replace("0 hours", "12 hours").replace("wengtengjin", "10 hours").replace("60 minutes", "59 minutes").replace("(180, 190) degrees", "(170, 180) degrees").replace("(10 hours and 10 minutes, 10 hours and 20 minutes)", "(10 hours and 05 minutes, 10 hours and 15 minutes)")
                answer = an["answer"].replace("10 hours", "wengtengjin").replace("0 hours", "12 hours").replace("wengtengjin", "10 hours").replace("60 minutes", "59 minutes").replace("(180, 190) degrees", "(170, 180) degrees").replace("(10 hours and 10 minutes, 10 hours and 20 minutes)", "(10 hours and 05 minutes, 10 hours and 15 minutes)")
                an['option'] = option_text
                an['answer'] = answer
                number_value = number_key[cls].pop()
                an["id"] = f"{number_value}_{cls}"
                if "(e)" in an["option"]:
                    an["option_num"] = 5
                elif "(d)" in an["option"]:
                    an["option_num"] = 4
                else:
                    an["option_num"] = 3
                an["text"] = an["text"].replace("image", "figure").replace("diagram", "figure").replace("following color of", "following colors of")
                current_dir = os.path.join(target_dir, cls)
                if os.path.exists(current_dir) == False:
                    os.makedirs(current_dir)
                file_name = (str(number_value) + "_" + cls + an["image_path"][-4:]).replace("jpg","png")
    
                

                target_file = os.path.join(current_dir, file_name)
                an["image_path"] ="/".join(target_file.replace("\\","/").split("/")[2:])[25:]
                number_value += 1
                all_json_list.append(an)
                resize_image(source_image, target_file)  # 使用 copy2 保留文件的元数据（如创建时间等）
                # all_data.extend(data)  # 将列表中的元素逐一添加到 all_data
                            
# 提取所有 Major_categories
major_categories = [item['Major_categories'] for item in all_json_list]
counter = 1  # 初始化计数器
task_categories = []

for item in all_json_list:
    # 使用 get 获取 'task_class'，如果没有则返回 'none' + 计数器
    task_class = item.get('task_class', f"none")
    
    # 如果是 'none' 开头的值，则计数器加一
    if task_class.startswith('none'):
        counter += 1
        
    task_categories.append(task_class)


# 初始化计数器：以 'Major_categories' 和 'task_class' 为键
category_task_counts = defaultdict(lambda: defaultdict(int))

for item in all_json_list:
    major_category = item.get('Major_categories')
    task_class = item.get('task_class', 'none')  # 默认值为 'none'，如果没有 'task_class'
    
    # 增加 'task_class' 的计数
    category_task_counts[major_category][task_class] += 1
    
    
    
# 使用 Counter 统计种类和对应的数量
category_counts = Counter(major_categories)
task_counts = Counter(task_categories)


# 输出结果
for category, count in category_counts.items():
    print(f"{category}: {count}")
    
data = {
    "type":file_names[index],
    "total": len(all_json_list),
    "category_counts":category_counts,
    "task_counts":task_counts,
    "details":category_task_counts,
    "data": all_json_list
    }
# 导出为 JSON 文件
with open(output_path, 'w') as json_file:
    json.dump(data, json_file, indent=4)


print("所有符合条件的文件已成功复制！")


/Users/wtj/Desktop/Number_Sense/Only_data
/Users/wtj/Desktop/
Directory '/Users/wtj/Desktop/Number_Sense/Only_data/Real-World Dataset' exists. Deleted all contents.
Directory '/Users/wtj/Desktop/Number_Sense/Only_data/Real-World Dataset' has been created.
/Users/wtj/Desktop/Number_Sense/Real-World Dataset
Scale: 143
Depth: 154
Volume: 147
Angle: 149
Quantity: 147
Length: 162
所有符合条件的文件已成功复制！


In [4]:
#beifen，不shuffle的做法
import shutil
import json
from PIL import Image
import os
import shutil
import random
from collections import Counter
random.seed(42)
from collections import defaultdict

pwd = os.getcwd().replace("\\","/")
print(pwd)

"""
Synthetic Mathematical Dataset
Real-World Dataset
"""
r = 0
index = 0
file_names = ["Synthetic Mathematical Dataset", "Real-World Dataset"]
json_names = ["/Synthetic.json", "/Real.json"]
file_name = file_names[index]
json_name= json_names[index]

sou = "/".join(pwd.split("/")[:5]) + "/"
im = "/".join(pwd.split("/")[:4]) + "/"
print(im)

# 定义源目录和目标目录
source_dir = sou + file_name
# target_dir = pwd + "/" + file_name + "/images"
target_dir = pwd + "/" + file_name

output_path = pwd + "/" + file_name + json_name

# 检查目标目录是否存在
if os.path.exists(target_dir):
    # 删除目标目录及其所有内容
    shutil.rmtree(target_dir)
    print(f"Directory '{target_dir}' exists. Deleted all contents.")
# 创建一个新的空目录
os.makedirs(target_dir)
print(f"Directory '{target_dir}' has been created.")

def resize_image(source_file, target_file):
    # 打开图片
    with Image.open(source_file) as img:
                # Convert the image to RGB mode
        img = img.convert("RGB")
        # 获取图片的尺寸 (宽度, 高度)
        w, h = img.size
        
        # 如果图片的宽度或高度大于500，进行缩放
        if w > 500 or h > 500:
            # 计算缩放比例
            if w > h:
                new_w = 500
                new_h = int((500 / w) * h)
            else:
                new_h = 500
                new_w = int((500 / h) * w)
            
            # 缩放图片
            img = img.resize((new_w, new_h), Image.Resampling.LANCZOS)
        
        # 保存到新路径
        img.save(target_file)
        
all_json_list = []
number = 0
all_data = []
number_dict = {"Length": 0,
        "Quantity": 0,
        "Angle": 0,
        "Volume": 0,
        "Scale": 0,
        "Depth": 0,
        "Area": 0}
print(source_dir)
# 遍历源目录及子目录
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if file.endswith(('.json')):
            # 获取源文件路径
            source_file = os.path.join(root, file)
            if "init_image" in source_file:
                continue
            with open(source_file, 'r', encoding='utf-8') as file:
                data = json.load(file)
            for an in data:
                cls = an["image_path"].split("/")[2].split("_")[0]
                if cls != "Length":
                    continue
                
                # if cls != "Depth":
                #     continue
                an["Major_categories"] = cls
                an["Minor_category"] = an["image_path"].split("/")[3]
                #hours
                option_text = an["option"].replace("10 hours", "wengtengjin").replace("0 hours", "12 hours").replace("wengtengjin", "10 hours")
                answer = an["answer"].replace("10 hours", "wengtengjin").replace("0 hours", "12 hours").replace("wengtengjin", "10 hours")
                an['option'] = option_text
                an['answer'] = answer

                
                an["id"] = f"{number_dict[cls]}_{cls}"
                if "(e)" in an["option"]:
                    an["option_num"] = 5
                elif "(d)" in an["option"]:
                    an["option_num"] = 4
                else:
                    an["option_num"] = 3
                an["text"] = an["text"].replace("image", "figure").replace("diagram", "figure")
                current_dir = os.path.join(target_dir, cls)
                if os.path.exists(current_dir) == False:
                    os.makedirs(current_dir)
                file_name = str(number_dict[cls]) + "_" + cls + an["image_path"][-4:]
    
                
                source_image = os.path.join(im, an["image_path"])
                # print(source_image)
                if not os.path.exists(source_image):
                    continue
                target_file = os.path.join(current_dir, file_name)
                an["image_path"] ="/".join(target_file.replace("\\","/").split("/")[2:])[25:]
                number_dict[cls] += 1
                all_json_list.append(an)
                resize_image(source_image, target_file)  # 使用 copy2 保留文件的元数据（如创建时间等）
                # all_data.extend(data)  # 将列表中的元素逐一添加到 all_data
                            
# 提取所有 Major_categories
major_categories = [item['Major_categories'] for item in all_json_list]
counter = 1  # 初始化计数器
task_categories = []

for item in all_json_list:
    # 使用 get 获取 'task_class'，如果没有则返回 'none' + 计数器
    task_class = item.get('task_class', f"none")
    
    # 如果是 'none' 开头的值，则计数器加一
    if task_class.startswith('none'):
        counter += 1
        
    task_categories.append(task_class)


# 初始化计数器：以 'Major_categories' 和 'task_class' 为键
category_task_counts = defaultdict(lambda: defaultdict(int))

for item in all_json_list:
    major_category = item.get('Major_categories')
    task_class = item.get('task_class', 'none')  # 默认值为 'none'，如果没有 'task_class'
    
    # 增加 'task_class' 的计数
    category_task_counts[major_category][task_class] += 1
    
    
    
# 使用 Counter 统计种类和对应的数量
category_counts = Counter(major_categories)
task_counts = Counter(task_categories)


# 输出结果
for category, count in category_counts.items():
    print(f"{category}: {count}")
    
data = {
    "type":file_names[index],
    "total": len(all_json_list),
    "category_counts":category_counts,
    "task_counts":task_counts,
    "details":category_task_counts,
    "data": all_json_list
    }
# 导出为 JSON 文件
with open(output_path, 'w') as json_file:
    json.dump(data, json_file, indent=4)


print("所有符合条件的文件已成功复制！")


/Users/wtj/Desktop/Number_Sense/Only_data
/Users/wtj/Desktop/
Directory '/Users/wtj/Desktop/Number_Sense/Only_data/Synthetic Mathematical Dataset' exists. Deleted all contents.
Directory '/Users/wtj/Desktop/Number_Sense/Only_data/Synthetic Mathematical Dataset' has been created.
/Users/wtj/Desktop/Number_Sense/Synthetic Mathematical Dataset
Length: 181
所有符合条件的文件已成功复制！


In [17]:
number_dict = {"Length": 0,
        "Quantity": 0,
        "Angle": 0,
        "Volume": 0,
        "Scale": 0,
        "Depth": 0,
        "Area": 0}
# 遍历源目录及子目录
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if file.endswith(('.json')):
            # 获取源文件路径
            source_file = os.path.join(root, file)
            if "init_image" in source_file:
                continue
            with open(source_file, 'r', encoding='utf-8') as file:
                data = json.load(file)
            for an in data:
                cls = an["image_path"].split("/")[2].split("_")[0]
                source_image = os.path.join(im, an["image_path"])
                if not os.path.exists(source_image):
                    continue
                number_dict[cls] += 1

# 将字典的值转换为随机顺序的数组
for key in number_dict:
    # 创建一个从 0 到 number_dict[key] 的列表
    array = list(range(number_dict[key]))  # 注意这里要加1，因为 range 不包含结束值
    # 打乱列表
    random.shuffle(array)
    # 将打乱后的列表赋值给字典的值
    number_dict[key] = array
print(len(number_dict["Depth"]))

   
                

100


In [14]:
import json
from collections import defaultdict

json_path = "Real-World Dataset/Real.json"
with open(json_path, 'r', encoding='utf-8') as file:
    data = json.load(file)
    
    # 替换 'your_class_key' 为实际字段名（例如: 'class'）
    category_counts = defaultdict(int)
    for item in data["data"]:
        class_name = item.get("Major_categories")
        if class_name != "Volume":
            continue
        class_name = item.get("Minor_category")  # 修改这里为实际字段名
        category_counts[class_name] += 1
    
    # 输出统计结果
    for cls, count in category_counts.items():
        print(f"类别 '{cls}' 的出现次数: {count}")
    print("\n总计类别数量:", len(category_counts))

类别 'bottle_water_compare' 的出现次数: 11
类别 'box-google-three' 的出现次数: 7
类别 'box-bottle' 的出现次数: 41
类别 'box-multi' 的出现次数: 45
类别 'box-google -four' 的出现次数: 3
类别 'box-max' 的出现次数: 40

总计类别数量: 6
